In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name())
from tqdm import tqdm
import re

In [2]:
# HTS
t18 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding='cp949')
t19 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding='cp949')
t20 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding='cp949')
t21 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding='cp949')

# 경기일정
sc = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv', encoding='cp949')

# 선수
p18 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding='cp949')
p19 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding='cp949')
p20 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding='cp949')
p21 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding='cp949')

# 타자 기본
h18 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2018.csv', encoding='cp949')
h19 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2019.csv', encoding='cp949')
h20 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2020.csv', encoding='cp949')
h21 = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2021.csv', encoding='cp949')

# 팀
team = pd.read_csv('data/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv', encoding='cp949')

# 서브미션
submission = pd.read_excel('data/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx')

In [3]:
hts = pd.concat([t18, t19, t20, t21], axis=0).reset_index(drop=True)
player = pd.concat([p18, p19, p20, p21], axis=0).reset_index(drop=True)
hit = pd.concat([h18, h19, h20, h21], axis=0).reset_index(drop=True)

In [4]:
hts.columns = ['연도', '경기코드', '타구코드', '선수코드', '팀코드', '이닝', '타구속도', '발사각도', '타격결과', '상대투수투구구속', '해당구장']
player.columns = ['연도', '선수코드', '선수명', '팀코드', '포지션', '나이', '연봉']
hit.columns = ['연도', '선수코드', '경기', '타석', '타수', '타율', '안타', '홈런', '루타', '장타율', '희생플라이', '볼넷', '삼진', '고의4구', '사구', '병살타']

In [5]:
player = player[-player['연봉'].isnull()]
player['money2'] = player['연봉'].str.replace('[가-힣]','').astype('int64')
player.loc[player['연봉'].str.endswith('달러'), 'money2'] = player.loc[player['연봉'].str.endswith('달러'), 'money2'].apply(lambda x : x * 0.12).astype('int64')
player['연봉'] = player['money2']
player.drop('money2', axis=1, inplace=True)
data = {'연도': 2020, '선수코드': 50802, '선수명': '화이트', '팀코드':'SK', '포지션':'내', '나이':31, '연봉':16000}
player = player.append(data, ignore_index=True)
hts.shape, player.shape, hit.shape

((120745, 11), (2635, 7), (1098, 16))

In [6]:
hts['상대팀코드'] = hts.경기코드.str[-5:-1]
for i in tqdm(range(hts.shape[0])):
    hts['상대팀코드'].iloc[i]=hts.iloc[i].상대팀코드.replace(hts.iloc[i].팀코드, '')

100%|█████████████████████████████████████████████████████████████████████████| 120745/120745 [06:43<00:00, 298.91it/s]


In [9]:
hts['날짜'] = '20' + hts.타구코드.str.split('_').str[0]
hts['날짜'] = pd.to_datetime(hts.날짜)
hts['월'] = hts.날짜.dt.month
hts['일'] = hts.날짜.dt.day
hts['시간'] = hts.날짜.dt.hour
hts['요일'] = hts.날짜.dt.dayofweek

In [10]:
hts['분기'] = pd.cut(hts['월'], bins = [1, 4, 7, 10, 13],
                   right = False, labels = [1, 2, 3, 4]).astype('category')

hts['주_그룹'] = pd.cut(hts['일'], bins = [1, 10, 20, 32],
                     right = False, labels = ['월초', '월중', '월말'])

hts['주말여부'] = hts['요일'].apply(lambda x : 1 if x>=5 else 0)

In [11]:
# 출루율 = (안타 + 볼넷 + 몸에 맞는 볼) / (타수 + 볼넷 + 몸에 맞는 볼 + 희생플라이)
# on base percentage
hit['출루율'] = (hit['안타'] + hit['볼넷'] + hit['사구']) / (hit['타수'] + hit['볼넷'] + hit['사구'] + hit['희생플라이'])
hit['OPS'] = hit['장타율'] + hit['출루율']
team.columns = ['연도', '팀코드', '팀이름']
hit = hit.merge(player, on=['연도', '선수코드'], how='left')
hit = hit.merge(team, on=['연도', '팀코드'], how='left')

### hit_log

In [12]:
hit_log = pd.read_csv('data/hit_log.csv', index_col=0) # 타자 당 날짜별 기록
hit_log['날짜'] = hit_log['연도'].astype(int).astype(str) + '-' + hit_log['ss'].astype(str)
hit_log['상대팀코드'] = hit_log['상대'].apply(lambda x : 'KT' if ('KT' in x)
                                       else 'LG' if ('LG' in x)
                                       else 'NC' if ('NC' in x)
                                       else 'HT' if ('KIA' in x)
                                       else 'SK' if (('SK' in x) or ('SSG' in x))
                                       else 'SS' if ('삼성' in x)
                                       else 'LT' if ('롯데' in x)
                                       else 'OB' if ('두산' in x)
                                       else 'HH' if ('한화' in x)
                                       else 'WO')
hit_log['스코어'] = hit_log['결과'].apply(lambda x : x.split(' ')[1]) 
hit_log['결과'] = hit_log['결과'].apply(lambda x : x.split(' ')[0])

In [13]:
hts = hts.merge(player.drop('팀코드', axis=1), on=['선수코드', '연도'], how='left')
hts['날짜'] = hts['날짜'].astype(str)

In [14]:
hit_log['날짜'] = pd.to_datetime(hit_log.날짜)
hit_log['월'] = hit_log.날짜.dt.month
hit_log['일'] = hit_log.날짜.dt.day
hit_log['요일'] = hit_log.날짜.dt.dayofweek
hit_log['날짜'] = hit_log['날짜'].astype(str)

In [15]:
hit_log['분기'] = pd.cut(hit_log['월'], bins = [1, 4, 7, 10, 13],
                   right = False, labels = [1, 2, 3, 4]).astype('category')

hit_log['주_그룹'] = pd.cut(hit_log['일'], bins = [1, 10, 20, 32],
                     right = False, labels = ['월초', '월중', '월말'])

hit_log['주말여부'] = hit_log['요일'].apply(lambda x : 1 if x>=5 else 0)

In [16]:
hit_log = hit_log.query('날짜 <= "2021-07-11"')

In [17]:
hts1 = hts[['상대팀코드', '팀코드', '날짜']].drop_duplicates()
hit_log = hit_log.merge(hts1, on=['날짜', '상대팀코드'], how='left')

player = player.query('포지션 != "투"')
hit_log = hit_log.merge(player, on=['연도', '팀코드', '선수명'], how='left')

In [18]:
hit_log = hit_log.fillna(method='ffill')

In [19]:
hit_log

,ss,상대,결과,타순,P,선발,타수,득점,안타,2타,...,일,요일,분기,주_그룹,주말여부,팀코드,선수코드,포지션,나이,연봉
0,07-19,@KT,L,8,PH,0,1,0,1,0,...,19,3,3,월중,0,HH,60184.0,포,27.0,2700.0
1,07-22,@삼성,L,8,C,1,3,0,0,0,...,22,6,3,월말,1,HH,60184.0,포,27.0,2700.0
2,09-06,@KT,W,5,PR,0,2,1,1,1,...,6,3,3,월초,0,HH,60184.0,포,27.0,2700.0
3,09-07,@KT,L,6,PH,0,1,0,0,0,...,7,4,3,월초,0,HH,60184.0,포,27.0,2700.0
4,09-09,@LG,L,8,PH,0,2,0,0,0,...,9,6,3,월초,1,HH,60184.0,포,27.0,2700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63743,07-04,한화,W,6,1B,1,3,1,1,0,...,4,6,3,월초,1,LG,69102.0,내,20.0,3000.0
63744,07-05,한화,W,2,1B,1,4,1,2,0,...,5,0,3,월초,0,LG,69102.0,내,20.0,3000.0
63745,07-02,@KT,L,8,1B,1,3,0,1,0,...,2,4,3,월초,0,WO,69366.0,내,21.0,3000.0
63746,07-04,@KT,L,8,1B,1,2,0,0,0,...,4,6,3,월초,1,WO,69366.0,내,21.0,3000.0


### HTS

In [20]:
hts18 = hts.query('연도 == 2018')
hts19 = hts.query('연도 == 2019')
hts20 = hts.query('연도 == 2020')
hts21 = hts.query('연도 == 2021')

hts18 = hts18.query('날짜 <= "2018-10-14"')
hts19 = hts19.query('날짜 <= "2019-10-01"')
hts20 = hts20.query('날짜 <= "2020-10-31"')
hts21 = hts21.query('날짜 <= "2021-07-11"')

In [21]:
hts_tr_18 = hts18.query('날짜 < "2018-09-19"')
hts_ts_18 = hts18.query('날짜 >= "2018-09-19"')

hts_tr_19 = hts19.query('날짜 < "2019-09-11"')
hts_ts_19 = hts19.query('날짜 >= "2019-09-11"')

hts_tr_20 = hts20.query('날짜 < "2020-10-08"')
hts_ts_20 = hts20.query('날짜 >= "2020-10-08"')

hts_21 = hts21

### 데이터 셋 생성

In [22]:
# feature : 시즌 별 마지막 21경기 제외하고 성적
# target : 시즌 별 마지막 21경기 ops
# 18년 : ~ 9/19 ~ 10/14
# 19년 : ~ 9/11 ~ 10/1
# 20년 : ~ 10/8 ~ 10/31
# 21년 : ~ 9/15 ~ 10/8

In [23]:
hl18 = hit_log.query('연도 == 2018')
hl19 = hit_log.query('연도 == 2019')
hl20 = hit_log.query('연도 == 2020')
hl21 = hit_log.query('연도 == 2021')   # 데이터 불완전

hl18 = hl18.query('날짜 <= "2018-10-14"')
hl19 = hl19.query('날짜 <= "2019-10-01"')
hl20 = hl20.query('날짜 <= "2020-10-31"')
hl21 = hl21.query('날짜 <= "2021-07-11"')

In [24]:
# test는 출루율, 장타율, ops를 구하기 위한 것
train_18 = hl18.query('날짜 < "2018-09-19"')
test_18 = hl18.query('날짜 >= "2018-09-19"')

train_19 = hl19.query('날짜 < "2019-09-11"')
test_19 = hl19.query('날짜 >= "2019-09-11"')

train_20 = hl20.query('날짜 < "2020-10-08"')
test_20 = hl20.query('날짜 >= "2020-10-08"')

train_21 = hl21

In [25]:
def train_set(train, test, hts):
    
    # train과 test 둘 다 있는 선수 뽑아내기
    pl = list(set(train['선수코드'].unique()) & set(test['선수코드'].unique()))
    
    ### hit_log
    agg_dict = {
        '선발':['sum'],
        '타수':['sum', 'mean'],
        '득점':['sum', 'mean'],
        '안타':['sum', 'mean'],
        '2타':['sum', 'mean'],
        '3타':['sum', 'mean'],
        '홈런':['sum', 'mean'],
        '루타':['sum', 'mean', 'std'],
        '타점':['sum', 'mean', 'std'],
        '도루':['sum', 'mean'],
        '도실':['sum', 'mean'],
        '볼넷':['sum', 'mean'],
        '사구':['sum', 'mean'],
        '고4':['sum', 'mean'],
        '삼진':['sum', 'mean'],
        '병살':['sum', 'mean'],
        '희타':['sum', 'mean'],
        '희비':['sum', 'mean'],
        '타율':['mean', 'std'],
        '출루':['sum', 'mean'],
        '장타':['sum', 'mean'],
        '투구':['sum', 'mean', 'max', 'std'],
        'avLl':['mean', 'std'],
        'RE24':['mean', 'std'],
        'WPA':['mean', 'std'],
    }
    
    f = train.groupby('선수코드').agg(agg_dict)
    f = f.fillna(0)
    f.columns = [('_').join(column).upper() for column in f.columns.ravel()]
    f = f.reset_index()
    f = f.loc[f['선수코드'].isin(pl)]
    f['안타/타수'] = f['안타_SUM'] / f['타수_SUM']
    f['2타/타수'] = f['2타_SUM'] / f['타수_SUM']
    f['3타/타수'] = f['3타_SUM'] / f['타수_SUM']
    f['홈런/타수'] = f['홈런_SUM'] / f['타수_SUM']
    f['루타/타수'] = f['루타_SUM'] / f['타수_SUM']
    f['투구/타수'] = f['투구_SUM'] / f['타수_SUM']
    
    # 상대팀 코드별
    f1 = train.groupby(['선수코드', '상대팀코드']).agg(agg_dict).unstack()
    f1 = f1.fillna(0)
    f1.columns = ['상대팀별_'+ str(column) for column in f1.columns]
    f1 = f1.reset_index()
    f1 = f1.loc[f1['선수코드'].isin(pl)]
    
    # 월별
    f2 = train.groupby(['선수코드', '월']).agg(agg_dict).unstack()
    f2 = f2.fillna(0)
    f2.columns = ['월별_'+ str(column) for column in f2.columns]
    f2 = f2.reset_index()
    f2 = f2.loc[f2['선수코드'].isin(pl)]
    
    # 요일별
    f3 = train.groupby(['선수코드', '요일']).agg(agg_dict).unstack()
    f3 = f3.fillna(0)
    f3.columns = ['요일별_'+ str(column) for column in f3.columns]
    f3 = f3.reset_index()
    f3 = f3.loc[f3['선수코드'].isin(pl)]
    
    # 분기별
    f4 = train.groupby(['선수코드', '분기']).agg(agg_dict).unstack()
    f4 = f4.fillna(0)
    f4.columns = ['분기별_'+ str(column) for column in f4.columns]
    f4 = f4.reset_index()
    f4 = f4.loc[f4['선수코드'].isin(pl)]
    
    # 주_그룹별
    f5 = train.groupby(['선수코드', '주_그룹']).agg(agg_dict).unstack()
    f5 = f5.fillna(0)
    f5.columns = ['주_그룹별_'+ str(column) for column in f5.columns]
    f5 = f5.reset_index()
    f5 = f5.loc[f5['선수코드'].isin(pl)]
    
    # 주말 여부별
    f6 = train.groupby(['선수코드', '주말여부']).agg(agg_dict).unstack()
    f6 = f6.fillna(0)
    f6.columns = ['주말여부별_'+ str(column) for column in f6.columns]
    f6 = f6.reset_index()
    f6 = f6.loc[f6['선수코드'].isin(pl)]
    
    
    ### HTS
    agg_dict = {
        '타구속도':['mean', 'max', 'std', 'skew'],
        '발사각도':['mean', 'max', 'std', 'skew'],
    }
    
    f7 = hts.groupby('선수코드').agg(agg_dict)
    f7 = f7.fillna(0)
    f7.columns = [('_').join(column).upper() for column in f7.columns.ravel()]
    f7 = f7.reset_index()
    f7 = f7.loc[f7['선수코드'].isin(pl)]
    
    # 상대팀 코드별
    f8 = hts.groupby(['선수코드', '상대팀코드']).agg(agg_dict).unstack()
    f8 = f8.fillna(0)
    f8.columns = ['상대팀별_'+ str(column) for column in f8.columns]
    f8 = f8.reset_index()
    f8 = f8.loc[f8['선수코드'].isin(pl)]
    
    # 구장별
    f9 = hts.groupby(['선수코드', '해당구장']).agg(agg_dict).unstack()
    f9 = f9.fillna(0)
    f9.columns = ['구장별_'+ str(column) for column in f9.columns]
    f9 = f9.reset_index()
    f9 = f9.loc[f9['선수코드'].isin(pl)]

    # 시간별
    f10 = hts.groupby(['선수코드', '해당구장']).agg(agg_dict).unstack()
    f10 = f10.fillna(0)
    f10.columns = ['시간별_'+ str(column) for column in f10.columns]
    f10 = f10.reset_index()
    f10 = f10.loc[f10['선수코드'].isin(pl)]
    
    # 요일별
    f11 = hts.groupby(['선수코드', '해당구장']).agg(agg_dict).unstack()
    f11 = f11.fillna(0)
    f11.columns = ['요일별_'+ str(column) for column in f11.columns]
    f11 = f11.reset_index()
    f11 = f11.loc[f11['선수코드'].isin(pl)]
    
    # 분기별
    f12 = hts.groupby(['선수코드', '해당구장']).agg(agg_dict).unstack()
    f12 = f12.fillna(0)
    f12.columns = ['분기별_'+ str(column) for column in f12.columns]
    f12 = f12.reset_index()
    f12 = f12.loc[f12['선수코드'].isin(pl)]
    
    # 주_그룹별
    f13 = hts.groupby(['선수코드', '해당구장']).agg(agg_dict).unstack()
    f13 = f13.fillna(0)
    f13.columns = ['주_그룹별_'+ str(column) for column in f13.columns]
    f13 = f13.reset_index()
    f13 = f13.loc[f13['선수코드'].isin(pl)]
    
    # 주말여부별
    f14 = hts.groupby(['선수코드', '해당구장']).agg(agg_dict).unstack()
    f14 = f14.fillna(0)
    f14.columns = ['주말여부별_'+ str(column) for column in f14.columns]
    f14 = f14.reset_index()
    f14 = f14.loc[f14['선수코드'].isin(pl)]
        
    
    
    ### 출루율, 장타율, OPS 
    base = pd.DataFrame()
    
    # 출루율 파트
    AB = pd.DataFrame(train.groupby('선수코드')['타수'].sum())
    H = pd.DataFrame(train.groupby('선수코드')['안타'].sum() + train.groupby('선수코드')['2타'].sum() + train.groupby('선수코드')['3타'].sum() + train.groupby('선수코드')['홈런'].sum())
    BB = pd.DataFrame(train.groupby('선수코드')['볼넷'].sum())
    HBP = pd.DataFrame(train.groupby('선수코드')['사구'].sum())
    SF = pd.DataFrame(train.groupby('선수코드')['희타'].sum())
    
    # 장타율 파트
    H1 = pd.DataFrame(train.groupby('선수코드')['안타'].sum())
    H2 = pd.DataFrame(train.groupby('선수코드')['2타'].sum())
    H3 = pd.DataFrame(train.groupby('선수코드')['3타'].sum())
    HR = pd.DataFrame(train.groupby('선수코드')['홈런'].sum())
    
    base = AB.merge(H, on='선수코드')
    base = base.merge(BB, on='선수코드')
    base = base.merge(HBP, on='선수코드')
    base = base.merge(SF, on='선수코드')
    base = base.merge(H1, on='선수코드')
    base = base.merge(H2, on='선수코드')
    base = base.merge(H3, on='선수코드')
    base = base.merge(HR, on='선수코드')
    
    base.columns = ['타수', '총안타', '볼넷', '사구', '희생타', '안타', '2타', '3타', '홈런']
    
    base['출루율_tr'] = (base['총안타'] + base['볼넷'] + base['사구']) / (base['타수'] + base['볼넷'] + base['사구'] + base['희생타'])
    base['장타율_tr'] = (base['안타'] + 2*base['2타'] + 3*base['3타'] + 4*base['홈런']) / base['타수']
    base['OPS_tr'] = base['출루율_tr'] + base['장타율_tr']
    
    base = base.reset_index()
    base = base.loc[base['선수코드'].isin(pl)]
    
    
    
    ### merge
    f = f.merge(f1, on='선수코드', how='left')
    f = f.merge(f2, on='선수코드', how='left')
    f = f.merge(f3, on='선수코드', how='left')
    f = f.merge(f4, on='선수코드', how='left')
    f = f.merge(f5, on='선수코드', how='left')
    f = f.merge(f6, on='선수코드', how='left')
    f = f.merge(f7, on='선수코드', how='left')
    f = f.merge(f8, on='선수코드', how='left')
    f = f.merge(f9, on='선수코드', how='left')
    f = f.merge(f10, on='선수코드', how='left')
    f = f.merge(f11, on='선수코드', how='left')
    f = f.merge(f12, on='선수코드', how='left')
    f = f.merge(f13, on='선수코드', how='left')
    f = f.merge(f14, on='선수코드', how='left')
    f = f.merge(base[['선수코드', '출루율_tr', '장타율_tr', 'OPS_tr']], on='선수코드', how='left')

    return f

In [26]:
train18 = train_set(train_18, test_18, hts_tr_18)
train19 = train_set(train_19, test_19, hts_tr_19)
train20 = train_set(train_20, test_20, hts_tr_20)
train21 = train_set(train_21, train_21, hts_21)

In [32]:
ftr = pd.concat([train18, train19], axis=0)
ftr = ftr.reset_index(drop=True)

In [33]:
ftr.head()

,선수코드,선발_SUM,타수_SUM,타수_MEAN,득점_SUM,득점_MEAN,안타_SUM,안타_MEAN,2타_SUM,2타_MEAN,...,"주_그룹별_('발사각도', 'std', '창원')","주_그룹별_('발사각도', 'skew', '창원')","주말여부별_('타구속도', 'mean', '창원')","주말여부별_('타구속도', 'max', '창원')","주말여부별_('타구속도', 'std', '창원')","주말여부별_('타구속도', 'skew', '창원')","주말여부별_('발사각도', 'mean', '창원')","주말여부별_('발사각도', 'max', '창원')","주말여부별_('발사각도', 'std', '창원')","주말여부별_('발사각도', 'skew', '창원')"
0,60184.0,1,9,1.800000,1,0.200000,2,0.400000,1,0.200000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60343.0,35,151,2.013333,18,0.240000,35,0.466667,4,0.053333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60523.0,24,130,1.830986,23,0.323944,36,0.507042,7,0.098592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60558.0,75,309,2.686957,47,0.408696,81,0.704348,16,0.139130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60566.0,32,181,1.691589,30,0.280374,47,0.439252,2,0.018692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
X_test = train20.drop('선수코드', axis=1).fillna(0)

In [37]:
f_list = list(set(ftr.columns) & set(X_test.columns))
ftr = ftr.loc[:, f_list]
X_test = X_test.loc[:, f_list]

In [38]:
ftr.shape, X_test.shape

((380, 2172), (200, 2172))

In [39]:
def test_set(train, test):
    
    # train과 test 둘 다 있는 선수 뽑아내기
    pl = list(set(train['선수코드'].unique()) & set(test['선수코드'].unique()))
    
    # 출루율 파트
    AB = pd.DataFrame(test.groupby('선수코드')['타수'].sum())
    H = pd.DataFrame(test.groupby('선수코드')['안타'].sum() + test.groupby('선수코드')['2타'].sum() + test.groupby('선수코드')['3타'].sum() + test.groupby('선수코드')['홈런'].sum())
    BB = pd.DataFrame(test.groupby('선수코드')['볼넷'].sum())
    HBP = pd.DataFrame(test.groupby('선수코드')['사구'].sum())
    SF = pd.DataFrame(test.groupby('선수코드')['희타'].sum() + test.groupby('선수코드')['희비'].sum())
    
    # 장타율 파트
    H1 = pd.DataFrame(test.groupby('선수코드')['안타'].sum())
    H2 = pd.DataFrame(test.groupby('선수코드')['2타'].sum())
    H3 = pd.DataFrame(test.groupby('선수코드')['3타'].sum())
    HR = pd.DataFrame(test.groupby('선수코드')['홈런'].sum())
    
    test = AB.merge(H, on='선수코드')
    test = test.merge(BB, on='선수코드')
    test = test.merge(HBP, on='선수코드')
    test = test.merge(SF, on='선수코드')
    test = test.merge(H1, on='선수코드')
    test = test.merge(H2, on='선수코드')
    test = test.merge(H3, on='선수코드')
    test = test.merge(HR, on='선수코드')
    
    test.columns = ['타수', '총안타', '볼넷', '사구', '희생타', '안타', '2타', '3타', '홈런']
    
    test['출루율'] = (test['총안타'] + test['볼넷'] + test['사구']) / (test['타수'] + test['볼넷'] + test['사구'] + test['희생타'])
    test['장타율'] = (test['안타'] + 2*test['2타'] + 3*test['3타'] + 4*test['홈런']) / test['타수']
    test['OPS'] = test['출루율'] + test['장타율']
    
    test = test.reset_index()
    test = test.loc[test['선수코드'].isin(pl)]
    test = test.reset_index(drop=True)
    
    return test[['선수코드', '출루율', '장타율', 'OPS']]

In [40]:
test18 = test_set(train_18, test_18).fillna(0)
test19 = test_set(train_19, test_19).fillna(0)
test20 = test_set(train_20, test_20).fillna(0)

In [41]:
target = pd.concat([test18, test19], axis=0)
target = target.reset_index(drop=True)

In [42]:
y_test = test20

In [43]:
X_test.shape, y_test.shape

((200, 2172), (200, 4))

### 데이터 오버샘플링 - SMOGN

In [44]:
# 회귀문제를 위한 오버샘플링
import smogn

In [45]:
data1 = pd.concat([ftr, target['출루율']], axis=1)
data2 = pd.concat([ftr, target['장타율']], axis=1)

data1 = data1.fillna(0)
data2 = data2.fillna(0)

In [46]:
## conduct smogn
data_smogn1 = smogn.smoter(
    ## main arguments
    data = data1,             ## pandas dataframe
    y = '출루율',              ## string ('header name')
    k = 7,                    ## positive integer (k < n)
    pert = 0.04,              ## real number (0 < R < 1)
    samp_method = 'balance',  ## string ('balance' or 'extreme')
    drop_na_col = True,       ## boolean (True or False)
    drop_na_row = True,       ## boolean (True or False)
    replace = False,          ## boolean (True or False)
)

r_index: 100%|#########################################################################| 25/25 [00:04<00:00,  5.61it/s]


In [47]:
## conduct smogn
data_smogn2 = smogn.smoter(
    ## main arguments
    data = data2,             ## pandas dataframe
    y = '장타율',             ## string ('header name')
    k = 7,                    ## positive integer (k < n)
    pert = 0.04,              ## real number (0 < R < 1)
    samp_method = 'balance',  ## string ('balance' or 'extreme')
    drop_na_col = True,       ## boolean (True or False)
    drop_na_row = True,       ## boolean (True or False)
    replace = False,          ## boolean (True or False)
)

r_index: 100%|###########################################################################| 2/2 [00:00<00:00, 10.39it/s]


In [48]:
data_smogn1 = data_smogn1.reset_index(drop=True)
data_smogn2 = data_smogn2.reset_index(drop=True)

In [49]:
dataset1 = pd.concat([data1, data_smogn1], axis=0).reset_index(drop=True)
dataset2 = pd.concat([data2, data_smogn2], axis=0).reset_index(drop=True)

In [50]:
dataset1.shape, dataset2.shape

((718, 2173), (713, 2173))

### 모델링
- 출루율, 장타율 따로 fitting 해줌. 나중에 합쳐서 OPS를 만듦

In [52]:
X1 = dataset1.drop(['출루율'], axis=1).values
y1 = dataset1.출루율

X2 = dataset2.drop(['장타율'], axis=1).values
y2 = dataset2.장타율

In [53]:
X1.shape, y1.shape, X2.shape, y2.shape

((718, 2172), (718,), (713, 2172), (713,))

In [54]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, random_state = 607, shuffle = True)

In [55]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [56]:
from lightgbm import LGBMRegressor

In [57]:
lgbm = LGBMRegressor(random_state = 607, max_depth = 4, n_estimators = 20000, learning_rate = .01)
lgbm_pred_1 = np.zeros((X_test.shape[0]))
mape_list = []
for tr_idx, val_idx in kf.split(X1):
    tr_x, val_x = X1[tr_idx], X1[val_idx]
    tr_y, val_y = y1.iloc[tr_idx], y1.iloc[val_idx]
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], eval_metric = 'mean_absolute_percentage_error', early_stopping_rounds = 500, verbose = 2000)
    pred = lgbm.predict(val_x)
    mape = MAPE(val_y, pred)
    mape_list.append(mape)
    print(f'FOLD MAPE = {mape}')
    sub_pred = lgbm.predict(X_test) / 5
    lgbm_pred_1 += sub_pred
print(f'\n{lgbm.__class__.__name__} MAPE = {np.mean(mape_list)}')

Training until validation scores don't improve for 500 rounds
[2000]	training's mape: 0.0111776	training's l2: 0.000382327	valid_1's mape: 0.0624167	valid_1's l2: 0.0101513
[4000]	training's mape: 0.00371935	training's l2: 0.000113502	valid_1's mape: 0.0569758	valid_1's l2: 0.00993872
Early stopping, best iteration is:
[4534]	training's mape: 0.0030884	training's l2: 0.000102108	valid_1's mape: 0.0565013	valid_1's l2: 0.00993221
FOLD MAPE = inf
Training until validation scores don't improve for 500 rounds
[2000]	training's mape: 0.0107357	training's l2: 0.000331834	valid_1's mape: 0.0572894	valid_1's l2: 0.00963884
[4000]	training's mape: 0.0033875	training's l2: 0.000111136	valid_1's mape: 0.0525992	valid_1's l2: 0.00947216
Early stopping, best iteration is:
[5071]	training's mape: 0.00226892	training's l2: 9.86104e-05	valid_1's mape: 0.0517351	valid_1's l2: 0.00945617
FOLD MAPE = inf
Training until validation scores don't improve for 500 rounds
[2000]	training's mape: 0.0111638	train

In [58]:
lgbm = LGBMRegressor(random_state = 607, max_depth = 4, n_estimators = 20000, learning_rate = .01)
lgbm_pred_2 = np.zeros((X_test.shape[0]))
mape_list = []
for tr_idx, val_idx in kf.split(X2):
    tr_x, val_x = X2[tr_idx], X2[val_idx]
    tr_y, val_y = y2.iloc[tr_idx], y2.iloc[val_idx]
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], eval_metric = 'mean_absolute_percentage_error', early_stopping_rounds = 500, verbose = 2000)
    pred = lgbm.predict(val_x)
    mape = MAPE(val_y, pred)
    mape_list.append(mape)
    print(f'FOLD MAPE = {mape}')
    sub_pred = lgbm.predict(X_test) / 5
    lgbm_pred_2 += sub_pred
print(f'\n{lgbm.__class__.__name__} MAPE = {np.mean(mape_list)}')

Training until validation scores don't improve for 500 rounds
[2000]	training's mape: 0.0126289	training's l2: 0.000461378	valid_1's mape: 0.0809464	valid_1's l2: 0.0175883
Early stopping, best iteration is:
[3379]	training's mape: 0.00548238	training's l2: 0.000132043	valid_1's mape: 0.0758005	valid_1's l2: 0.0174594
FOLD MAPE = inf
Training until validation scores don't improve for 500 rounds
[2000]	training's mape: 0.0151041	training's l2: 0.000582526	valid_1's mape: 0.0748666	valid_1's l2: 0.017844
[4000]	training's mape: 0.00582553	training's l2: 0.00014267	valid_1's mape: 0.0666239	valid_1's l2: 0.0174716
[6000]	training's mape: 0.00298025	training's l2: 7.88838e-05	valid_1's mape: 0.0639318	valid_1's l2: 0.0173344
[8000]	training's mape: 0.00180924	training's l2: 6.44572e-05	valid_1's mape: 0.0628766	valid_1's l2: 0.0172776
[10000]	training's mape: 0.00129998	training's l2: 6.05534e-05	valid_1's mape: 0.0624151	valid_1's l2: 0.0172618
[12000]	training's mape: 0.000998423	trainin

In [59]:
from ngboost import NGBRegressor

In [60]:
ngb = NGBRegressor(n_estimators = 15000, verbose = 0, random_state = 607)
ngb_pred_1 = np.zeros((X_test.shape[0]))
mape_list = []
for tr_idx, val_idx in kf.split(X1):
    tr_x, val_x = X1[tr_idx], X1[val_idx]
    tr_y, val_y = y1.iloc[tr_idx], y1.iloc[val_idx]
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 2000)
    pred = ngb.predict(val_x)
    mape = MAPE(val_y, pred)
    mape_list.append(mape)
    print(f'FOLD MAPE = {mape}')
    sub_pred = ngb.predict(X_test) / 5
    ngb_pred_1 += sub_pred
print(f'\n{ngb.__class__.__name__} MAPE = {np.mean(mape_list)}')

FOLD MAPE = inf
FOLD MAPE = inf
FOLD MAPE = inf
FOLD MAPE = inf
FOLD MAPE = inf

NGBRegressor MAPE = inf


In [61]:
ngb = NGBRegressor(n_estimators = 15000, verbose = 0, random_state = 607)
ngb_pred_2 = np.zeros((X_test.shape[0]))
mape_list = []
for tr_idx, val_idx in kf.split(X2):
    tr_x, val_x = X2[tr_idx], X2[val_idx]
    tr_y, val_y = y2.iloc[tr_idx], y2.iloc[val_idx]
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 2000)
    pred = ngb.predict(val_x)
    mape = MAPE(val_y, pred)
    mape_list.append(mape)
    print(f'FOLD MAPE = {mape}')
    sub_pred = ngb.predict(X_test) / 5
    ngb_pred_2 += sub_pred
print(f'\n{ngb.__class__.__name__} MAPE = {np.mean(mape_list)}')

FOLD MAPE = inf
FOLD MAPE = inf
FOLD MAPE = inf
FOLD MAPE = inf
FOLD MAPE = inf

NGBRegressor MAPE = inf


In [62]:
from sklearn.ensemble import RandomForestRegressor

In [63]:
rf = RandomForestRegressor(n_estimators = 15000, verbose = 0, random_state = 607)
rf_pred_1 = np.zeros((X_test.shape[0]))
mape_list = []
for tr_idx, val_idx in kf.split(X1):
    tr_x, val_x = X1[tr_idx], X1[val_idx]
    tr_y, val_y = y1.iloc[tr_idx], y1.iloc[val_idx]
    rf.fit(tr_x, tr_y)
    pred = rf.predict(val_x)
    mape = MAPE(val_y, pred)
    mape_list.append(mape)
    print(f'FOLD MAPE = {mape}')
    sub_pred = rf.predict(X_test) / 5
    rf_pred_1 += sub_pred
print(f'\n{rf.__class__.__name__} MAPE = {np.mean(mape_list)}')

KeyboardInterrupt: 

In [ ]:
rf = RandomForestRegressor(n_estimators = 15000, verbose = 0, random_state = 607)
rf_pred_2 = np.zeros((X_test.shape[0]))
mape_list = []
for tr_idx, val_idx in kf.split(X2):
    tr_x, val_x = X2[tr_idx], X2[val_idx]
    tr_y, val_y = y2.iloc[tr_idx], y2.iloc[val_idx]
    rf.fit(tr_x, tr_y)
    pred = rf.predict(val_x)
    mape = MAPE(val_y, pred)
    mape_list.append(mape)
    print(f'FOLD MAPE = {mape}')
    sub_pred = rf.predict(X_test) / 5
    rf_pred_2 += sub_pred
print(f'\n{rf.__class__.__name__} MAPE = {np.mean(mape_list)}')

In [64]:
lpred1 = pd.DataFrame(lgbm_pred_1, columns=['출루율_l'])
lpred2 = pd.DataFrame(lgbm_pred_2, columns=['장타율_l'])
sub = pd.concat([train20, lpred1, lpred2], axis=1)

In [65]:
npred1 = pd.DataFrame(ngb_pred_1, columns=['출루율_n'])
npred2 = pd.DataFrame(ngb_pred_2, columns=['장타율_n'])
sub = pd.concat([sub, npred1, npred2], axis=1)

In [ ]:
rpred1 = pd.DataFrame(rf_pred_1, columns=['출루율_r'])
rpred2 = pd.DataFrame(rf_pred_2, columns=['장타율_r'])
sub = pd.concat([sub, rpred1, rpred2], axis=1)

In [66]:
sub['출루율'] = 0.5*sub['출루율_l'] + 0.5*sub['출루율_n']
sub['장타율'] = 0.5*sub['장타율_l'] + 0.5*sub['장타율_n']

### 서브미션 파일

In [67]:
submission = pd.read_excel('data/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx')
submission.loc[0, 'OPS'] = 1.097
submission.PCODE[1]=76232;submission.PCODE[2]=68050;submission.PCODE[3]=75847;submission.PCODE[4]=67341;submission.PCODE[5]=79192
submission.PCODE[6]=78224;submission.PCODE[7]=78513;submission.PCODE[8]=76290;submission.PCODE[9]=79215;submission.PCODE[10]=67872;
submission = submission.iloc[1:-1, :]

In [68]:
submission.columns = ['NO.', '선수코드', 'OPS', '장타율', '출루율']
submission = submission.drop(['장타율', '출루율'], axis=1)
submission = submission.merge(sub[['선수코드', '출루율', '장타율']], on='선수코드', how='left')
submission['OPS'] = submission['출루율'] + submission['장타율']

In [69]:
lst = ['양의지', '강백호', '최정', '이정후', '채은성', '김재환', '전준우', '김현수', '박건우', '로맥']
submission['선수명'] = lst

In [70]:
y_test = y_test.loc[y_test['선수코드'].isin(submission['선수코드'])]

In [71]:
sub = submission.sort_values(by='선수코드', ascending=True).reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [72]:
mape_list1 = []
for i in range(0, 10):
    mape = MAPE(y_test['출루율'][i], sub['출루율'][i])
    mape_list1.append(mape)

In [73]:
mape_list2 = []
for i in range(0, 10):
    mape = MAPE(y_test['장타율'][i], sub['장타율'][i])
    mape_list2.append(mape)

In [74]:
mape_list3 = []
for i in range(0, 10):
    mape = MAPE(y_test['OPS'][i], sub['OPS'][i])
    mape_list3.append(mape)

In [75]:
mape_df = pd.DataFrame([mape_list1, mape_list2, mape_list3]).T
mape_df.columns = ['출루율_mape', '장타율_mape', 'OPS_mape']
mape_df

,출루율_mape,장타율_mape,OPS_mape
0,15.779022,42.184758,28.541553
1,31.302824,40.840990,37.118779
2,24.626395,12.327207,18.097774
3,25.838000,26.073727,25.971634
4,17.172172,46.726340,35.776353
5,11.470703,46.260070,28.052550
6,17.123912,24.278547,21.266069
7,5.643072,20.640627,10.335222
8,9.585232,25.916229,19.379869
9,28.029982,2.681269,14.931379


In [76]:
mape_df.mean()

출루율_mape    18.657131
장타율_mape    28.792976
OPS_mape    23.947118
dtype: float64

In [77]:
sub

,NO.,선수코드,OPS,출루율,장타율,선수명
0,4,67341,1.004712,0.467569,0.537142,이정후
1,10,67872,1.012837,0.431811,0.581026,로맥
2,2,68050,0.985771,0.425639,0.560132,강백호
3,3,75847,0.972976,0.422153,0.550823,최정
4,1,76232,0.926066,0.442505,0.483561,양의지
5,8,76290,1.007472,0.458997,0.548475,김현수
6,6,78224,0.934965,0.414380,0.520585,김재환
7,7,78513,0.936794,0.432755,0.504039,전준우
8,5,79192,0.963352,0.432418,0.530934,채은성
9,9,79215,0.947851,0.387531,0.560320,박건우


In [78]:
y_test

,선수코드,출루율,장타율,OPS
0,67341.0,0.403846,0.377778,0.781624
1,67872.0,0.628571,0.982143,1.610714
2,68050.0,0.564706,0.638889,1.203595
3,75847.0,0.569231,0.745098,1.314329
4,76232.0,0.534247,0.907692,1.441939
5,76290.0,0.411765,0.375000,0.786765
6,78224.0,0.500000,0.687500,1.187500
7,78513.0,0.409639,0.635135,1.044774
8,79192.0,0.478261,0.716667,1.194928
9,79215.0,0.538462,0.575758,1.114219
